# __Movie Sentiment Analysis__

In [144]:
import pandas as pd
import numpy as np
import re
import contractions
import nltk
import contractions
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import text_preprocessing as tp
import utils
import spacy


from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from flashtext import KeywordProcessor



warnings.filterwarnings("ignore")

## Importing Dataset for Movie Metadata

Use the IMDB dataset containing information about movies. Two different data sources will be utilized in the process of traning the model.

__a. Data Source:__ https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/

In [145]:
# Open file containing movie metadata
movies_metadata_df = pd.read_csv("./dataset/movies_metadata.csv")
movies_metadata_df.head(5)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,10/30/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,12/15/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,12/22/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,12/22/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,2/10/1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [146]:
# Open file containing the movie credits, i.e. casts and crews
movies_credits_df = pd.read_csv("./dataset/credits.csv")
movies_credits_df.head(5)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [147]:
# Combine the dataframes of movie metadata and credits
movies_metadata_df = pd.merge(movies_metadata_df, movies_credits_df, on=["id"], how="left")
movies_metadata_df.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [148]:
# List all the columns in movie metadata's dataframe
movies_metadata_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [149]:
# Drop columns that are not relevant
del_columns = ['belongs_to_collection', 'homepage', 'id', 'original_language', 'original_title', 'popularity', 'poster_path', 'adult',
               'overview', 'tagline', 'production_countries', 'spoken_languages', 'status', 'video', 'vote_average', 'vote_count']
movies_metadata_df = movies_metadata_df.drop(del_columns, axis=1)

# Remove 'tt' appended on the IMDB ID
movies_metadata_df['imdb_id'] = movies_metadata_df['imdb_id'].apply(lambda x : x[2:] if isinstance(x, str) and "tt" in x else x)

movies_metadata_df.head(5)

,budget,genres,imdb_id,production_companies,release_date,revenue,runtime,title,cast,crew
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",0114709,"[{'name': 'Pixar Animation Studios', 'id': 3}]",10/30/1995,373554033.0,81.0,Toy Story,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",0113497,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",12/15/1995,262797249.0,104.0,Jumanji,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0113228,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",12/22/1995,0.0,101.0,Grumpier Old Men,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",0114885,[{'name': 'Twentieth Century Fox Film Corporat...,12/22/1995,81452156.0,127.0,Waiting to Exhale,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,0,"[{'id': 35, 'name': 'Comedy'}]",0113041,"[{'name': 'Sandollar Productions', 'id': 5842}...",2/10/1995,76578911.0,106.0,Father of the Bride Part II,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


__b. Data Source:__ https://www.kaggle.com/datasets/raynardj/imdb-vision-and-nlp/data

In [150]:
# Open another file containing the movie metadata
movies_df = pd.read_csv("./dataset/movies.csv")
movies_df.head(5)

,imdbId,Imdb Link,Title,IMDB Score,Genre,local_image_path
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation;Adventure;Comedy,images/114709_.jpg
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action;Adventure;Family,images/113497_.jpg
2,113277,http://www.imdb.com/title/tt113277,Heat (1995),8.2,Action;Crime;Drama,images/113277_.jpg
3,114319,http://www.imdb.com/title/tt114319,Sabrina (1995),6.3,Comedy;Drama,images/114319_.jpg
4,114576,http://www.imdb.com/title/tt114576,Sudden Death (1995),5.7,Action;Crime;Thriller,images/114576_.jpg


In [151]:
# Rename the column names of movies info by removing the spaces and converting to lower case
movies_df = movies_df.rename(columns={"imdbId" : "imdb_id", "Imdb Link" : "imbd_link", "Title" : "movie", "IMDB Score" : "imdb_score", "Genre" : "genre"})
print(movies_df.columns)

Index(['imdb_id', 'imbd_link', 'movie', 'imdb_score', 'genre',
       'local_image_path'],
      dtype='object')


In [152]:
# Drop columns that are not relevant
del_columns = ['imbd_link', 'imdb_score', 'local_image_path']

movies_df = movies_df.drop(columns=del_columns, axis=1)
movies_df.head(5)

,imdb_id,movie,genre
0,114709,Toy Story (1995),Animation;Adventure;Comedy
1,113497,Jumanji (1995),Action;Adventure;Family
2,113277,Heat (1995),Action;Crime;Drama
3,114319,Sabrina (1995),Comedy;Drama
4,114576,Sudden Death (1995),Action;Crime;Thriller


In [153]:
# Merge two different datasets for movies metadata

movies_metadata_df['imdb_id'] = movies_metadata_df['imdb_id'].fillna(0) # fixed errors on NaN values when converting to int data type
movies_metadata_df['imdb_id'] = movies_metadata_df['imdb_id'].astype(int)

movies_metadata_df = pd.merge(movies_metadata_df, movies_df, on=["imdb_id"], how="right")
movies_metadata_df.head(5)

,budget,genres,imdb_id,production_companies,release_date,revenue,runtime,title,cast,crew,movie,genre
0,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",114709,"[{'name': 'Pixar Animation Studios', 'id': 3}]",10/30/1995,373554033.0,81.0,Toy Story,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",Toy Story (1995),Animation;Adventure;Comedy
1,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",113497,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",12/15/1995,262797249.0,104.0,Jumanji,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",Jumanji (1995),Action;Adventure;Family
2,60000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",113277,"[{'name': 'Regency Enterprises', 'id': 508}, {...",12/15/1995,187436818.0,170.0,Heat,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",Heat (1995),Action;Crime;Drama
3,58000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",114319,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...",12/15/1995,0.0,127.0,Sabrina,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",Sabrina (1995),Comedy;Drama
4,35000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",114576,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",12/22/1995,64350171.0,106.0,Sudden Death,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",Sudden Death (1995),Action;Crime;Thriller


## Importing Dataset for Movie Reviews

Use the IMDB dataset for movie reviews.

__Data Source:__ https://www.kaggle.com/datasets/raynardj/imdb-vision-and-nlp/data

In [154]:
# Open the file containing the movie reviews
reviews_df = pd.read_csv("./dataset/reviews.csv")
reviews_df.head(5)

,review_id,reviewer,movie,rating,review_summary,review_date,spoiler_tag,review_detail,helpful,roberta_sentiment
0,rw0213367,SUPERNOVA HEIGHTS,A Nightmare on Elm Street (1984),8.0,A Classic of the horror films History,27-Aug-03,0,This is the beginning of a great horror film s...,"['0', '0']",Positive
1,rw0213369,kibler@adelphia.net,A Nightmare on Elm Street (1984),NaN,Better than your average horror movie,01-Sep-03,0,"Nightmare on Elm Street, A (1984) John Saxon, ...","['0', '0']",Negative
2,rw0213371,matthew87,A Nightmare on Elm Street (1984),NaN,good slasher flick,01-Sep-03,0,"the best freddy film period,1 because the horr...","['0', '0']",Positive
3,rw0213375,rossrobinson,A Nightmare on Elm Street (1984),10.0,A nightmare on elm st part 1,27-Sep-03,0,I remember seeing a nightmare on elm street pa...,"['0', '2']",Negative
4,rw0213376,Andres24,A Nightmare on Elm Street (1984),9.0,Highway to hell,03-Oct-03,0,"It's a nightmare. If Nancy falls asleep, they...","['1', '1']",Positive


In [155]:
# Merge the movie metadata (from the same kaggle source) and reviews
movie_reviews_df = pd.merge(movies_df, reviews_df, on=["movie"], how="inner")
movie_reviews_df.head(5)

,imdb_id,movie,genre,review_id,reviewer,rating,review_summary,review_date,spoiler_tag,review_detail,helpful,roberta_sentiment
0,114709,Toy Story (1995),Animation;Adventure;Comedy,rw1223346,KUAlum26,10.0,Can it be ten years already?,23-Nov-05,0,PIXAR animation was but an experimental animat...,"['1', '1']",Positive
1,114709,Toy Story (1995),Animation;Adventure;Comedy,rw3110243,MUFCOK,10.0,A childhood favourite,24-Oct-14,0,Toy Story is one of those movies which holds a...,"['1', '2']",Positive
2,114709,Toy Story (1995),Animation;Adventure;Comedy,rw3109622,Rainey-Dawn,6.0,Not As Good As I Was Hoping,23-Oct-14,0,"This is another one of my ""I should like this ...","['2', '8']",Neutral
3,114709,Toy Story (1995),Animation;Adventure;Comedy,rw3124846,byson5186,9.0,Why The First Toy Story Is My Favorite,15-Nov-14,0,So far they have come out with 3 Toy Story mov...,"['0', '1']",Positive
4,114709,Toy Story (1995),Animation;Adventure;Comedy,rw3635799,snperera,9.0,A classic friendship story!,08-Feb-17,0,"Wow, well what can I say! This is a favorite o...","['0', '1']",Positive


In [156]:
# Drop columns that are not relevant, including the movie and genre. IMDB ID can be used to associate 
# review records with movie metadata
del_columns = ['movie', 'genre', 'review_id', 'reviewer', 'spoiler_tag', 'helpful', 'rating']

movie_reviews_df = movie_reviews_df.drop(columns=del_columns, axis=1)
movie_reviews_df.head(5)

,imdb_id,review_summary,review_date,review_detail,roberta_sentiment
0,114709,Can it be ten years already?,23-Nov-05,PIXAR animation was but an experimental animat...,Positive
1,114709,A childhood favourite,24-Oct-14,Toy Story is one of those movies which holds a...,Positive
2,114709,Not As Good As I Was Hoping,23-Oct-14,"This is another one of my ""I should like this ...",Neutral
3,114709,Why The First Toy Story Is My Favorite,15-Nov-14,So far they have come out with 3 Toy Story mov...,Positive
4,114709,A classic friendship story!,08-Feb-17,"Wow, well what can I say! This is a favorite o...",Positive


## Exploratory Data Analysis

Check and resolve duplicate records

In [157]:
# Drop duplicate records in movies metadata
print(f"Record count before removing duplicates => {movies_metadata_df.shape[0]}")
movies_metadata_df.drop_duplicates(inplace=True)
print(f"Record count after removing duplicates  => {movies_metadata_df.shape[0]}")

Record count before removing duplicates => 4174
Record count after removing duplicates  => 4071


In [158]:
# Drop duplicate records in movies reviews
print(f"Record count before removing duplicates => {movie_reviews_df.shape[0]}")
movie_reviews_df.drop_duplicates(inplace=True)
print(f"Record count after removing duplicates  => {movie_reviews_df.shape[0]}")

Record count before removing duplicates => 538381
Record count after removing duplicates  => 524520


Check and resolve missing values for `movie metadata`

In [159]:
# Check for missing values in movies metadata
movies_metadata_df.isna().sum()

budget                  3
genres                  3
imdb_id                 0
production_companies    3
release_date            3
revenue                 3
runtime                 3
title                   3
cast                    3
crew                    3
movie                   0
genre                   0
dtype: int64

In [160]:
# List of movies with missing information
movies_metadata_df[movies_metadata_df['title'].isna()]

,budget,genres,imdb_id,production_companies,release_date,revenue,runtime,title,cast,crew,movie,genre
2780,NaN,NaN,462322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grindhouse (2007),Action;Horror;Thriller
3521,NaN,NaN,448694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Puss in Boots (2011),Animation;Action;Adventure
3793,NaN,NaN,1937390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nymphomaniac: Vol. I (2013),Drama


In [161]:
# Drop all movies with missing information
movies_metadata_df.dropna(inplace=True, axis=0)
movies_metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4068 entries, 0 to 4173
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4068 non-null   float64
 1   genres                4068 non-null   object 
 2   imdb_id               4068 non-null   int64  
 3   production_companies  4068 non-null   object 
 4   release_date          4068 non-null   object 
 5   revenue               4068 non-null   float64
 6   runtime               4068 non-null   float64
 7   title                 4068 non-null   object 
 8   cast                  4068 non-null   object 
 9   crew                  4068 non-null   object 
 10  movie                 4068 non-null   object 
 11  genre                 4068 non-null   object 
dtypes: float64(3), int64(1), object(8)
memory usage: 413.2+ KB


Check and resolve missing values for `movie reviews`

In [162]:
# Check for missing values in movies reviews
movie_reviews_df.isna().sum()

imdb_id              0
review_summary       1
review_date          0
review_detail        0
roberta_sentiment    0
dtype: int64

In [163]:
# Replace all missing values in review summary and detail with a blank
movie_reviews_df[['review_summary', 'review_detail']] = movie_reviews_df[['review_summary', 'review_detail']].fillna('')
movie_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 524520 entries, 0 to 538380
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   imdb_id            524520 non-null  int64 
 1   review_summary     524520 non-null  object
 2   review_date        524520 non-null  object
 3   review_detail      524520 non-null  object
 4   roberta_sentiment  524520 non-null  object
dtypes: int64(1), object(4)
memory usage: 24.0+ MB


#### __Data Wrangling and Visualizations__:

Conduct data wrangling on `movie metadata` to transform and structure data into a desired format

In [164]:
# Separate movie release year from the title
pattern_year = r'\((\d{4})\)$'
movies_metadata_df['release_year'] = movies_metadata_df['movie'].str.extract(pattern_year).astype(int)
movies_metadata_df['movie'] = movies_metadata_df['movie'].str.replace(pattern_year, '', regex=True).str.strip()
movies_metadata_df[['movie', 'release_year']].head(5)

# Drop columns containing duplicate movie information
dup_columns = ['genres', 'release_date', 'title']
movies_metadata_df = movies_metadata_df.drop(columns=dup_columns, axis=1)

# Rearrange the order of columns
desired_order = ['imdb_id', 'movie', 'release_year', 'genre', 'runtime', 'budget', 'revenue', 'production_companies', 'cast', 'crew']
movies_metadata_df = movies_metadata_df[desired_order]

movies_metadata_df.head(5)

,imdb_id,movie,release_year,genre,runtime,budget,revenue,production_companies,cast,crew
0,114709,Toy Story,1995,Animation;Adventure;Comedy,81.0,30000000.0,373554033.0,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,113497,Jumanji,1995,Action;Adventure;Family,104.0,65000000.0,262797249.0,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,113277,Heat,1995,Action;Crime;Drama,170.0,60000000.0,187436818.0,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de..."
3,114319,Sabrina,1995,Comedy;Drama,127.0,58000000.0,0.0,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de..."
4,114576,Sudden Death,1995,Action;Crime;Thriller,106.0,35000000.0,64350171.0,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de..."


In [165]:
# Convert genre into list data type
movies_metadata_df['genre'] = movies_metadata_df['genre'].apply(lambda x : x.split(";"))

# Convert to a list of production companies
movies_metadata_df['production_companies'] = movies_metadata_df['production_companies'].apply(lambda x : utils.convert_str_to_dict(x, ['name']))
movies_metadata_df['production_companies'] = movies_metadata_df['production_companies'].apply(lambda list_of_dicts : [value for d in list_of_dicts for value in d.values()])

# TODO: Need to fix the issue on cast and crew with names containing single quotation
# movies_metadata_df['cast'] = movies_metadata_df['cast'].apply(lambda x : utils.convert_str_to_dict(x, ['character', 'name']))

movies_metadata_df[['genre', 'production_companies', 'cast', 'crew']].head(5)

,genre,production_companies,cast,crew
0,"[Animation, Adventure, Comedy]",[Pixar Animation Studios],"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,"[Action, Adventure, Family]","[TriStar Pictures, Teitler Film, Interscope Co...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,"[Action, Crime, Drama]","[Regency Enterprises, Forward Pass, Warner Bros.]","[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de..."
3,"[Comedy, Drama]","[Paramount Pictures, Scott Rudin Productions, ...","[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de..."
4,"[Action, Crime, Thriller]","[Universal Pictures, Imperial Entertainment, S...","[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de..."


Conduct data wrangling on `movie review` to transform and structure data into a desired format

In [166]:
# Get only the year the review was created
movie_reviews_df['review_year'] = pd.to_datetime(movie_reviews_df['review_date']).dt.year
movie_reviews_df = movie_reviews_df.drop(columns=['review_date'], axis=1)

movie_reviews_df.head(5)

,imdb_id,review_summary,review_detail,roberta_sentiment,review_year
0,114709,Can it be ten years already?,PIXAR animation was but an experimental animat...,Positive,2005
1,114709,A childhood favourite,Toy Story is one of those movies which holds a...,Positive,2014
2,114709,Not As Good As I Was Hoping,"This is another one of my ""I should like this ...",Neutral,2014
3,114709,Why The First Toy Story Is My Favorite,So far they have come out with 3 Toy Story mov...,Positive,2014
4,114709,A classic friendship story!,"Wow, well what can I say! This is a favorite o...",Positive,2017


Perform additional analysis of the data and create visual representations to gain a better understanding of the movie metadata and reviews. This information provides insights on trends and patterns in the movie data, which might be important especially on the reports that will be provided to the client.

In [167]:
# Show statistical summary of movie metadata and reviews
pd.merge(movies_metadata_df, movie_reviews_df, on=["imdb_id"], how="inner").describe()

,imdb_id,release_year,runtime,budget,revenue,review_year
count,5.244020e+05,524402.000000,524402.000000,5.244020e+05,5.244020e+05,524402.000000
mean,5.567062e+05,1995.708382,112.396383,4.177791e+07,1.530821e+08,2009.873698
std,7.600004e+05,18.310265,22.419268,5.306010e+07,2.429492e+08,6.264964
min,4.170000e+02,1902.000000,0.000000,0.000000e+00,0.000000e+00,1998.000000
25%,9.595600e+04,1988.000000,97.000000,3.500000e+06,1.029760e+07,2005.000000
50%,2.534740e+05,2002.000000,109.000000,2.100000e+07,6.127687e+07,2010.000000
75%,4.934640e+05,2008.000000,124.000000,6.000000e+07,1.861671e+08,2015.000000
max,5.247022e+06,2017.000000,254.000000,3.800000e+08,2.787965e+09,2021.000000


In [168]:
# TODO: Insert more visualizations
movie_reviews_df.head(3)




,imdb_id,review_summary,review_detail,roberta_sentiment,review_year
0,114709,Can it be ten years already?,PIXAR animation was but an experimental animat...,Positive,2005
1,114709,A childhood favourite,Toy Story is one of those movies which holds a...,Positive,2014
2,114709,Not As Good As I Was Hoping,"This is another one of my ""I should like this ...",Neutral,2014


In [169]:
import pandas as pd
from sklearn.model_selection import train_test_split


# Stratified sampling for creating a smaller sample
sample_size = 50000  # Adjust this based on your requirements
movie_reviews_df, _ = train_test_split(movie_reviews_df, train_size=sample_size, stratify=movie_reviews_df['roberta_sentiment'], random_state=42)

movie_reviews_df['roberta_sentiment'].value_counts()

Positive    23733
Neutral     14824
Negative    11443
Name: roberta_sentiment, dtype: int64

#### __For Text Pre-processing__:

__The movie summary and reviews undergo the following steps during the initial data pre-processing stage__
- Remove non-grammatical text like emails and URLs
- Replace non-ascii characters (some Python libraries are only limited to ascii characters like KeywordProcessor)
- Replace emojis with English word/s
- Handle contractions
- Handle slang words

In [170]:
emoji_dict = tp.get_emojis()
slang_word_dict = tp.get_slang_words(webscraped=False)

def initial_text_preprocessing(text):
    try:
        # Remove non-grammatical text
        text = tp.remove_email_address(text)
        text = tp.remove_hyperlink(text)

        # Replace non-ascii characters as there are Python libraries limiting this feature
        text = tp.replace_nonascii_characters(text)

        # Replace emojis with English word/s
        text = emoji_dict.replace_keywords(text)

        # Handle contractions
        text = contractions.fix(text)

        # Replace slang words
        text = slang_word_dict.replace_keywords(text)

    except Exception as err:
        print(f"ERROR: {err}")
        print(f"Input Text: {text}")

    return text


movie_reviews_df[['cleaned_review_summary', 'cleaned_review_detail']] = movie_reviews_df[['review_summary', 'review_detail']].applymap(initial_text_preprocessing)




In [171]:
movie_reviews_df.head()

,imdb_id,review_summary,review_detail,roberta_sentiment,review_year,cleaned_review_summary,cleaned_review_detail
76775,48356,One of my top-ten favorite movies,Marty (1955) was written by Paddy Chayefsky an...,Positive,2015,One of my top-ten favorite movies,Marty (1955) was written by Paddy Chayefsky an...
70790,119925,Mediocre Kostner Movie,The Postman isn't a bad movie but this movie d...,Negative,2010,Mediocre Kostner Movie,The Postman is not a bad movie but this movie ...
309153,62407,Informed and well-made look into marriage as a...,"Solid, incisive, non-linear look at the genesi...",Positive,2008,Informed and well-made look into marriage as a...,"Solid, incisive, non-linear look at the genesi..."
326648,367652,If you thought this movie was bad than your de...,I heard about this movie from a possible homos...,Negative,2006,If you thought this movie was bad than your de...,I heard about this movie from a possible homos...
498628,1877832,Pure fan indulgence with great production values,X-Men: Days of Future Past is certainly an amb...,Positive,2014,Pure fan indulgence with great production values,X-Men: Days of Future Past is certainly an amb...


In [172]:
# #Display the comparison between the top 5 original review summaries and their cleaned versions
# for i in range(1,movie_reviews_df.shape[0]):
#     print("Original Review Summary:")
#     print(movie_reviews_df['review_summary'][i])
#     print("\nCleaned Review Summary:")
#     print(movie_reviews_df['cleaned_review_summary'][i])
#     print("="*100)

# # Display the comparison between the top 5 original review details and their cleaned versions
# for i in range(1,movie_reviews_df.shape[0]):
#     print("Original Review Details:")
#     print(movie_reviews_df['review_detail'][i])
#     print("\nCleaned Review Details:")
#     print(movie_reviews_df['cleaned_review_detail'][i])
#     print("="*100)

__Name Entity Recognition (NER)__

In [173]:
# # TODO: Train own NER with the names of crews and casts and other important info

# # Load the English language model in spaCy
# nlp = spacy.load("en_core_web_sm")

# # Create a new dataFrame to store extracted name entitiesm then combine/rename the following labels
# # - LOCATION : GPE, LOC and FACE
# # - DATE_TIME : DATE, TIME
# # - ORGANIZATION : ORG

# column_names = ['PERSON', 'WORK_OF_ART', 'LOCATION', 'DATE_TIME', 'ORGANIZATION', 'PRODUCT', 'EVENT', 'LANGUAGE']
# movie_name_entities_df = pd.DataFrame(columns=column_names)

# # Extract and remove name entities from 'cleaned_review_summary' and 'cleaned_review_detail' columns
# movie_reviews_df['cleaned_review_summary'], movie_name_entities_df = tp.extract_name_entity(movie_reviews_df['cleaned_review_summary'], nlp, movie_name_entities_df)
# movie_reviews_df['cleaned_review_detail'], movie_name_entities_df = tp.extract_name_entity(movie_reviews_df['cleaned_review_detail'], nlp, movie_name_entities_df)

# # Display the comparison between the top 5 original review summaries and their cleaned versions
# for i in range(2):
#     print("Original Review Summary:")
#     print(movie_reviews_df['review_summary'][i])
#     print("\nCleaned Review Summary (NER Removed):")
#     print(movie_reviews_df['cleaned_review_summary'][i])
#     print("="*50)

# # Display the comparison between the top 5 original review details and their cleaned versions
# for i in range(2):
#     print("Original Review Details:")
#     print(movie_reviews_df['review_detail'][i])
#     print("\nCleaned Review Details (NER Removed):")
#     print(movie_reviews_df['cleaned_review_detail'][i])
#     print("="*50)

In [174]:
# Dataframe of the extracted name entities
#movie_name_entities_df = pd.merge(movie_reviews_df['imdb_id'], movie_name_entities_df, left_index=True, right_index=True)
#movie_name_entities_df.head(10)

__Utilize regular expressions to eliminate punctuation, substitute whitespaces, transform characters to lowercase, and filter out stopwords from movie reviews__

In [175]:
# Initialize the English stop words list
list_of_stopwords = set(stopwords.words('english'))

def supplemental_text_preprocessing(text):
    # Remove non-alphanumeric characters except for the following
    # - hyphen (-) that is in between alphanumeric
    text = tp.remove_non_alphanumeric_char(text)
    
    # Remove leading and trailing whitespaces
    text = text.strip()

    # Replace multiple whitespaces with a single space
    text = tp.replace_whitespace(text)

    # Remove stopwords
    text = tp.remove_stopwords(text, list_of_stopwords)

    return text

# Apply text cleaning to 'cleaned_review_summary' and 'cleaned_review_detail' columns
movie_reviews_df[['cleaned_review_summary', 'cleaned_review_detail']] = movie_reviews_df[['cleaned_review_summary', 'cleaned_review_detail']]\
                                                                            .applymap(supplemental_text_preprocessing)

# # Display the comparison between the top 5 original review summaries and their cleaned versions
# for i in range(2):
#     print("Original Review Summary:")
#     print(movie_reviews_df['review_summary'][i])
#     print("\nCleaned Review Summary (Non-alphanumeric Characters Removed, Lowercased, Extra Whitespaces Removed, Stopwords Removed):")
#     print(movie_reviews_df['cleaned_review_summary'][i])
#     print("="*100)

# # Display the comparison between the top 5 original review details and their cleaned versions
# for i in range(2):
#     print("Original Review Details:")
#     print(movie_reviews_df['review_detail'][i])
#     print("\nCleaned Review Details (Non-alphanumeric Characters Removed, Lowercased, Extra Whitespaces Removed, Stopwords Removed):")
#     print(movie_reviews_df['cleaned_review_detail'][i])
#     print("="*100)

__Implement Spacy's lemmatization on movie reviews, emphasizing the significance of lemmatization in capturing the base or root form of words__

In [176]:
# TODO: Words with hyphen are being tokenized or split

# Load the spaCy language model
# See: https://spacy.io/usage/models
nlp = spacy.load("en_core_web_sm")
    
# Apply lemmatization to 'review_summary' and 'review_detail' columns
movie_reviews_df['cleaned_review_summary'] = tp.lemmatize_text(movie_reviews_df['cleaned_review_summary'], nlp)
movie_reviews_df['cleaned_review_detail'] = tp.lemmatize_text(movie_reviews_df['cleaned_review_detail'], nlp)

# # Display the comparison between the top 5 original review summaries and their lemmatized versions
# for i in range(2):
#     print("Original Review Summary:")
#     print(movie_reviews_df['review_summary'][i])
#     print("\nLemmatized Review Summary:")
#     print(movie_reviews_df['cleaned_review_summary'][i])
#     print("="*100)

# # Display the comparison between the top 5 original review details and their lemmatized versions
# for i in range(2):
#     print("Original Review Details:")
#     print(movie_reviews_df['review_detail'][i])
#     print("\nLemmatized Review Details:")
#     print(movie_reviews_df['cleaned_review_detail'][i])
#     print("="*100)

In [177]:
movie_reviews_df.head(3)

movie_reviews_df.columns

Index(['imdb_id', 'review_summary', 'review_detail', 'roberta_sentiment',
       'review_year', 'cleaned_review_summary', 'cleaned_review_detail'],
      dtype='object')

## Model Creation

In [178]:
movie_reviews_df.columns

Index(['imdb_id', 'review_summary', 'review_detail', 'roberta_sentiment',
       'review_year', 'cleaned_review_summary', 'cleaned_review_detail'],
      dtype='object')

In [179]:
import pandas as pd

label_counts = movie_reviews_df['roberta_sentiment'].value_counts()

min_count = label_counts.min()

movie_reviews_df = pd.concat([movie_reviews_df[movie_reviews_df['roberta_sentiment'] == label].sample(min_count, random_state= 42) for label in label_counts.index])

movie_reviews_df = movie_reviews_df.sample(frac=1, random_state=42)



In [180]:
movie_reviews_df['roberta_sentiment'].value_counts()

Positive    11443
Negative    11443
Neutral     11443
Name: roberta_sentiment, dtype: int64

In [181]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
movie_reviews_df['roberta_sentiment'] = encoder.fit_transform(movie_reviews_df['roberta_sentiment'])
movie_reviews_df.head(5)

,imdb_id,review_summary,review_detail,roberta_sentiment,review_year,cleaned_review_summary,cleaned_review_detail
340121,427229,A real pleasant surprise!,I just could not see McConaughey and Parker to...,2,2006,real pleasant surprise,I McConaughey Parker romantic situation I imag...
217403,268380,Lame Brain Torture 5/10,"Unlike the enjoyable features Monsters, Inc. a...",0,2002,Lame Brain torture,unlike enjoyable feature Monsters Inc Shrek Ic...
88632,91149,Not one of my favorite Disney films.,The animation for this particular Disney outin...,0,2015,favorite Disney film,animation particular Disney outing opinion bit...
143317,125664,Carry as Carry,Jim Carry gives a hit or miss performance as A...,1,1999,carry Carry,Jim Carry hit miss performance Andy Kaufman hi...
220189,264935,Son of Leopold and Loeb,"This is an interesting crime drama, interestin...",0,2003,Son Leopold Loeb,interesting crime drama interesting miss explo...


In [182]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import accuracy_score, classification_report


x = movie_reviews_df['cleaned_review_summary'] + ' ' + movie_reviews_df['cleaned_review_detail']
y = movie_reviews_df['roberta_sentiment'].values





In [184]:

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=42)
vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)




In [ ]:


# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(y)


In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
# naive_bayes_classifier = MultinomialNB()
# naive_bayes_classifier.fit(X_train_tfidf, y_train)
# y_pred_mn = naive_bayes_classifier.predict(X_test_tfidf)

In [185]:
multinomial_nb_model = MultinomialNB()
multinomial_nb_model.fit(x_train, y_train)
predictions_multinomial = multinomial_nb_model.predict(x_test)


In [186]:
bernoulli_nb_model = BernoulliNB()
bernoulli_nb_model.fit(x_train, y_train)
predictions_bernoulli = bernoulli_nb_model.predict(x_test)


In [ ]:
# from sklearn.preprocessing import binarize


# X_train_tfidf = binarize(X_train_tfidf)
# X_test_tfidf = binarize(X_test_tfidf)

# bernoulli_naive_bayes = BernoulliNB()
# bernoulli_naive_bayes.fit(X_train_tfidf, y_train)

# y_pred_bnb = bernoulli_naive_bayes.predict(X_test_tfidf)

## Model Evaluation

In [187]:
accuracy_multinomial = accuracy_score(y_test, predictions_multinomial)
print("Multinomial Naive Bayes:")
print(f"Accuracy: {accuracy_multinomial:.2f}")
print(classification_report(y_test, predictions_multinomial))


Multinomial Naive Bayes:
Accuracy: 0.66
              precision    recall  f1-score   support

           0       0.65      0.74      0.69      2269
           1       0.63      0.54      0.58      2322
           2       0.70      0.72      0.71      2275

    accuracy                           0.66      6866
   macro avg       0.66      0.66      0.66      6866
weighted avg       0.66      0.66      0.66      6866



In [188]:
accuracy_bernoulli = accuracy_score(y_test, predictions_bernoulli)
print("\nBernoulli Naive Bayes:")
print(f"Accuracy: {accuracy_bernoulli:.2f}")
print(classification_report(y_test, predictions_bernoulli))



Bernoulli Naive Bayes:
Accuracy: 0.61
              precision    recall  f1-score   support

           0       0.67      0.51      0.58      2269
           1       0.61      0.54      0.58      2322
           2       0.58      0.78      0.66      2275

    accuracy                           0.61      6866
   macro avg       0.62      0.61      0.61      6866
weighted avg       0.62      0.61      0.61      6866



In [194]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

relevant_columns = ['cleaned_review_summary', 'cleaned_review_detail', 'roberta_sentiment']
df = movie_reviews_df[relevant_columns].dropna()

x= df[['cleaned_review_summary', 'cleaned_review_detail']]
y = df['roberta_sentiment'].values

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2,random_state=42)


vectorizer = CountVectorizer(max_features= 5000)
x_train_vectorized = vectorizer.fit_transform(x_train['cleaned_review_summary'] + ' ' + x_train['cleaned_review_detail']).toarray()
x_test_vectorized = vectorizer.transform(x_test['cleaned_review_summary'] + ' ' + x_test['cleaned_review_detail']).toarray()


logistic_regression_model = LogisticRegression()
# logistic_regression_model.fit(x_train_vectorized, y_train)
# predictions_logistic_regression = logistic_regression_model.predict(x_test_vectorized)


param_grid = {
    'C': [ 0.1, 1, 10],
    'penalty': ['l2']  
}


grid_search = GridSearchCV(logistic_regression_model, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(x_train_vectorized, y_train)


best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)


best_model = grid_search.best_estimator_
predictions_logistic_regression = best_model.predict(x_test_vectorized)



accuracy_logistic_regression = accuracy_score(y_test, predictions_logistic_regression)
print("Logistic Regression:")
print(f"Accuracy: {accuracy_logistic_regression:.2f}")
print(classification_report(y_test, predictions_logistic_regression))


Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Hyperparameters: {'C': 0.1, 'penalty': 'l2'}
Logistic Regression:
Accuracy: 0.69
              precision    recall  f1-score   support

           0       0.71      0.74      0.72      2269
           1       0.63      0.57      0.60      2322
           2       0.71      0.76      0.74      2275

    accuracy                           0.69      6866
   macro avg       0.69      0.69      0.69      6866
weighted avg       0.68      0.69      0.69      6866



In [ ]:
x_train_vectorized[0]

<1x48084 sparse matrix of type '<class 'numpy.int64'>'
	with 217 stored elements in Compressed Sparse Row format>

## References
- https://medium.com/analytics-vidhya/text-preprocessing-nlp-basics-430d54016048
- https://towardsdatascience.com/how-to-preprocess-social-media-data-and-text-messages-b011efacf74
- https://towardsdatascience.com/5-lesser-known-python-libraries-for-your-next-nlp-project-ff13fc652553
- https://medium.com/coinmonks/remaking-of-shortened-sms-tweet-post-slangs-and-word-contraction-into-sentences-nlp-7bd1bbc6fcff
- https://sunscrapers.com/blog/9-best-python-natural-language-processing-nlp/
- https://towardsdatascience.com/stemming-vs-lemmatization-2daddabcb221
- https://www.machinelearningplus.com/nlp/lemmatization-examples-python/#google_vignette
- https://atilaycemsamiloglu.medium.com/what-is-lemmatization-2d7a87f42c55
- https://support.prodi.gy/t/how-to-tell-spacy-not-to-split-any-intra-hyphen-words/1456
- https://towardsdatascience.com/elegant-text-pre-processing-with-nltk-in-sklearn-pipeline-d6fe18b91eb8
- https://python.plainenglish.io/optimize-your-spacy-ner-results-with-this-simple-change-e59937c411ab